# RCOT on SUTs (c<i)

This notebook starts from an exemplary supply-use table (SUT) that features fewer commodities than industries (c>i). The cases of [c=i](./SU-RCOT_c=i.ipynb) and [c>i](./SU-RCOT_c+i-.ipynb) are illustrated in separate notebooks. All notebooks share the same basic structure. That is, an illustrative SUT is imported from a spreadsheet; solved for the cases of factor constraints being absent or present; solved when new technologies are introduced; and compared to a solution relying on constructs.

Since the RCOT models allow for many degrees of freedom, certain aspects are highlighted only for either of the three SUT setups. For example, the issue of monetary or mixed commodity data is illustrated only in the [c=i](./SU-RCOT_c=i.ipynb) notebook. The overlapping parts, however, are explained in detail here and only briefly sketched out for the other two setups. Here, for the most part, we will look at the coefficient formulations of the RCOT models.

<a id='toc'></a>

The outline of the current notebook is as follows:


0. [Data import and set-up](#data-import)
1. [Empirical system in coefficent form w/out factor constraints](#emp_0f)\
    1.1 [As-is solution by inversion (use of constructs)](#emp_0f_inv)\
    1.2 [As-is solution via linear program (relative and absolute formulation)](#emp_0f_lp)\
2. [Empirical system in coefficient form with factor constraints](#emp_f_lp)\
    2.1 [Increasing final demand](#emp_f_lp_e+)\
    2.2 [Reducing final demand](#emp_f_lp_e-)\
    2.3 [Increasing factor availability](#emp_f_lp_f+)\
    2.4 [Reducing factor availability](#emp_f_lp_f-)
3. [Empirical system in absolute form with factor constraints](#emp_abs)
4. [Adding technology alternatives to the SUT system](#add_jp)\
    4.1 [An alternative for by-product production](#add_jp_bp)\
    4.2 [An alternative for main-product production with secondary products](#add_jp_1mp_jp)\
    4.3 [Multiple alternatives for main-product production](#add_jp_3mp_jp)\
    4.4 [Multiple alternatives for multiple technologies](#add_jp_nmp_sp-jp)    
5. [New alternatives for single production](#add_sp)

<a id='data-import'></a>

## 0. Data import and set-up

In [1]:
using LinearAlgebra
using JuMP
using GLPK
using XLSX

include("../src/SUT_structure.jl") # Used for the SUT setup <sut = SUT.structure(...)>
include("../src/Constructs.jl") # Used to derive single-production systems from the SUT setup, e.g. as <itc = Constructs.ITC(sut)>
include("../src/Auxiliary.jl") # Includes some helper functions
include("../src/RCOT_data.jl") # Sets up the data structure for RCOT modelling based on SUT.structure or Constructs.construct
include("../src/RCOT_model.jl"); # Builds and solves the RCOT models

In [2]:
# Load the workbook
xf = XLSX.readxlsx("../data/20230525_SUT_c-i+.xlsx")

XLSXFile("20230525_SUT_c-i+.xlsx") containing 4 Worksheets
            sheetname size          range        
-------------------------------------------------
                    V 6x4           A1:D6        
                  U+e 4x7           A1:G4        
                    F 5x6           A1:F5        
                   pi 5x2           A1:B5        


In [3]:
# Get data from all worksheets and convert it to float64 data type
V = convert(Matrix{Float64}, xf["V!B2:D6"])
U = convert(Matrix{Float64}, xf["U+e!B2:F4"])
e = convert(Matrix{Float64}, xf["U+e!G2:G4"])
F = convert(Matrix{Float64}, xf["F!B2:F5"])
pii = convert(Matrix{Float64}, xf["pi!B2:B5"])
t = xf["V!A1"];

As per the imported spreadsheets, $(V,U,e)$ are given in monetary units meaning that the commodity prices $p=i$. $F$ is in physical units and $\pi$ are the factor prices. As the system is rectangular (c<i), it cannot be solved for total output by inversion. Hence, either we solve via a linear program or we reallocate the by-products via constructs and thus allow for solution by inversion. For convenience, we pack the SUT system into an object that will be used in the following.

In [4]:
sut = SUT.structure(U,V,F,e,pii;t);

[ Info: A structure for supply-use elements was set up. Changing individual elements will not change others automatically.


In [5]:
# For completeness, let's check the total factor costs (in this unconstrained system)
sut.pii'*sut.f

1×1 Matrix{Float64}:
 123.99999999999999

Back to [table of contents](#toc)
<a id='emp_0f'></a>

## 1. Empirical system w/out factor constraint: solution by LP
Since the given SUT-system is again rectangular, we cannot perform imputation or impact analyses with it directly by inversion. Instead, we solve it again through an LP and by inversion after having applied constructs.

Back to [table of contents](#toc)
<a id='emp_0f_inv'></a>

### 1.1 Solution by inversion via constructs
Various constructs exits to reallocate by-products. Not all of them work for rectangular SUT-systems, though. For completeness, we introduce the Models A (commodity technology construct, CTC), B (industry technology construct, ITC), C (fixed industry sales structure construct, FISC), and D (fixed product sales structure construct, FPSC) here, although only two of them will be appropriate in the present case of more commodities than industries. We compute the solution here via the ITC and compare it to the commodity output $\bm{q}$ taken from the SUT.  We do the same for the total factor use $\bm{f}$.

In [6]:
itc = Constructs.ITC(sut);

In [7]:
# Compute the total output and compare it to the total commodity output; a tolerance measure is required to account for precision deviations
x = itc.L*itc.y
isapprox(x, sut.q, rtol = 1e-10)

true

In [8]:
# The corresponding factor use is calculated and compared to the empirical data; yet again a tolerance measure is required
f = itc.R*itc.L*itc.y
isapprox(f, sut.f, rtol = 1e-10)

true

Back to [table of contents](#toc)
<a id='emp_0f_lp'></a>

### 1.2 Solution by LP
Let us now solve the system via a linear program.

#### 1.2.1 Employing SUT coefficients

We start with a model in coefficient form.

In [9]:
# To set up our initial primal, we write:
primal = Model(GLPK.Optimizer)

# Define the optimisation model
@variable(primal, g_star[1:size(V,1)] >= 0, base_name = "g*")
@objective(primal, Min, sum(sut.pii'*sut.S*g_star))
@constraint(primal, c1, (sut.C-sut.B)*(g_star) .>= sut.e);

In [10]:
# Run the model and show results
optimize!(primal)
model_solution(primal)
@show value.(c1)
@show value.(g_star);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 124.0
value.(c1) = [51.0; 42.0; 31.0;;]
value.(g_star) = [16.925829775953638, 124.96554750130262, 122.35070431547483, 0.0, 0.0]


We can see in the output that the model found a feasible solution for both the primal and dual, with the resulting objective value being $124$, as we derived by calculating $\bm{\pi' f}$. However, the modelled industry output is different from the one we calculated directly from the SUT-system earlier. The reason for this is the combined effect of by-products being present and the system being rectangular without any factor constraints being present. It is particularly such factor constraints that enable the substitution/ complementation of technologies. In their absence, the system defaults back to a square outlay.

On the bright side, however, we can see that the shadow prices equal the unit vector; we should be reminded that $\bm{p = i}$ because the given system was in monetary values.

In [11]:
@show shadow_price.(c1);

shadow_price.(c1) = [-1.0; -1.0; -1.0;;]


Let us now model the dual explicitly and examine this in more detail:

In [12]:
# Set up the model
dual = Model(GLPK.Optimizer)

# Define the optimisation model
@variable(dual, p[1:size(sut.V,2)] >= 0)
@objective(dual, Max, sum(p'*sut.e))
@constraint(dual, c1d, (sut.C-sut.B)'*(p) .<= sut.S'*sut.pii);

In [13]:
# Run the model and show results
optimize!(dual)
model_solution(primal)
@show value.(p)
@show shadow_price.(c1d);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 124.0
value.(p) = [1.0, 1.0, 1.0]
shadow_price.(c1d) = [16.925829775953638; 124.96554750130262; 122.35070431547481; 0.0; 0.0;;]


While the model has a feasible solution for which the objective value is equal to that one of the primal and where the shadow prices reflect the originally observed (i.e. imported from spreadsheet) total output of the primal, we see indeed that the modelled price vector is equal the unit vector. On the other hand, however, the shadow prices for the dual, indicating the total output of the system, equal the solution vector $g*$ resulting from the primal.

Does that mean that the originally observed output vector $g$ is sub-optimal? No, as we can easily see when we calculate the dual's objective function and constraint separately with that vector:

In [14]:
println("The thus calculated objective value is: ", sut.pii'*sut.S*sut.g)
println("And the LHS of the constraint: ", (sut.C-sut.B)*sut.g)
println("And the same LHS with new prices: ", (sut.C-sut.B)*value.(g_star))

The thus calculated objective value is: [124.0;;]
And the LHS of the constraint: [51.0; 41.99999999999999; 31.000000000000014;;]
And the same LHS with new prices: [51.00000000000001, 42.0, 31.0]


With that, we see that a system consisting of more industries than commodities may have multiple alternate optimal solutions for its primal. That is, no unique solution for the output vector exists. The usefulness of the primal is hence limited in this case.

Mirroring the case of ([c>i](./SU-RCOT_c+i-.ipynb)), the reason for this behaviour is, mathematically speaking, the column-wise linear dependence of the net output (coefficients) matrix; it is thus not of full rank. Economically, market actors would produce initially in such a way that products with the highest output receive the highest ones
while other industries do not produce at all; in the present example, assuming that each industry is producing one primary product each next to potentially some secondary products, it is the primary products...

We encounter the opposite issue when more commodities are present than industries ([c>i](./SU-RCOT_c+i-.ipynb)), resulting in row-wise linear dependence of the net output matrix. That is, the dual will then reflect the empirical base data whereas the primal yields a non-unique solution.

In the following, we will concentrate on the primal.

#### 1.2.2 Absolute model formulation

Let us now model the primal in absolute form

In [15]:
# Instantiate the model
(@isdefined primal) && (primal = nothing; primal = Model(GLPK.Optimizer));

In [16]:
# To set up our initial primal, we write:
primal = Model(GLPK.Optimizer)

# Define the optimisation model
@variable(primal, z_star[1:size(V,1)] >= 0, base_name = "z*")
@objective(primal, Min, sum(sut.pii'*sut.F*z_star))
@constraint(primal, c1, (sut.V'-sut.U)*(z_star) .>= sut.e);

In [17]:
# Run the model and show results
optimize!(primal)
model_solution(primal)
@show value.(c1)
@show value.(z_star);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 124.0
value.(c1) = [51.0; 42.0; 31.0;;]
value.(z_star) = [0.15818532500891247, 1.2251524264833586, 1.3903489126758504, 0.0, 0.0]


Unsurprisingly, the absolute formulation yields the same result as the coefficient-based one above. That is, the objective value is the same and also the same industries are active. Multiplying the activity level by the original industry output gives the same modelled output as above.

In [18]:
isapprox(value.(g_star), value.(z_star).*sut.g, rtol = 1e-10) 

true

## 2. Empirical system with factor constraints: solution by LP
...

when factor constraints active, no such automatic choice occurs, 
but if no factor constraint active then a gradual shift towards a square solution occurs

In [19]:
su_rcot_data = RCOT_data.SU(sut);

┌ Info: You are setting up an SU-RCOT dataset. Elements of this dataset are now treated independently, 
└             meaning that no recalculation whatsoever takes place when individual elements are changed.


In [20]:
# Run the primal SU-RCOT in its absolute form
su_rcot_model = su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 123.99999999999999
value.(var_con) = [0.9999999999999993, 1.0, 1.0000000000000002, 0.9999999999999996, 0.9999999999999988]
value.(demand_con) = [51.0, 42.0, 31.0]
value.(factor_con) = [27.499999999999993, 6.666666666666666, 8.148148148148147, 21.428571428571427]


Let us lift the endowments of each factor by a smal percentage and see which industries are activated:

In [28]:
su_rcot_data.f = sut.f

4×1 Matrix{Float64}:
 27.499999999999996
  6.666666666666666
  8.148148148148147
 21.428571428571427

In [31]:
su_rcot_data.f = su_rcot_data.f*1.05

4×1 Matrix{Float64}:
 29.163749999999997
  7.069999999999999
  8.641111111111112
 22.725

In [32]:
# Run the primal SU-RCOT in its absolute form
su_rcot_model = su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 123.99999999999999
value.(var_con) = [0.2970079663940406, 1.2234173978690672, 1.2607000442865213, 0.0, 0.5566185493186068]
value.(demand_con) = [51.0, 42.0, 31.0]
value.(factor_con) = [24.38345811578551, 6.900524598514916, 8.213133965334693, 22.725]


We see that a different set of industries is activated.

In [ ]:
sensitivities(primal,type="constraint")

Row,name,value,rhs,slack,shadow_price,allowed_decrease,allowed_increase
,String,Float64,Float64,Float64,Float64,Float64,Float64
1,c1,57.0,57.0,0.0,-3.8004,-1.58046,1.38837
2,c1,79.0,79.0,0.0,-4.6216,-1.3886,1.2095
3,c1,34.9267,27.0,-7.92667,-0.0,-Inf,7.92667
4,c1,4.0,4.0,0.0,-6.9747,-1.06783,1.28671
5,c1,3.5,3.5,0.0,-14.7237,-0.562267,0.42194
6,c2,27.8,27.8,0.0,-18.4345,-0.248369,0.279814
7,c2,15.0203,16.1111,1.09085,0.0,-1.09085,Inf
8,c2,1.87152,2.22222,0.3507,0.0,-0.3507,Inf
9,c2,26.7604,30.4762,3.71584,0.0,-3.71584,Inf


Back to [table of contents](#toc)
<a id='add_jp'></a>

## 3. Adding technology alternatives under factor constraints
So far, we have only worked with the SUT system as imported and without considering factor constraints. Let us now add new technologies to it and see how substitution may or may not occur. We use the model in its absolute form.

Back to [table of contents](#toc)
<a id='add_jp_bp'></a>

### 3.1 Introducing an alternative single production technology

### 3.2 Introducing an alternative joint production technology

Let us first introduce an alternative to industry #i.3 (i.e. for producing commodity #c.1 as primary product), with no changed variables except the added technology.

In [19]:
# start from fresh data
su_rcot_data = RCOT_data.SU(sut);

┌ Info: You are setting up an SU-RCOT dataset. Elements of this dataset are now treated independently, 
└             meaning that no recalculation whatsoever takes place when individual elements are changed.


In [20]:
# Define supply and use of the additional technology
V3_alt1 = [78 5 0]
U3_alt1 = [2 12 48]'
F3_alt1 = [10 0 1.1 2.5]';

In [21]:
# Add the additional parameters to the copied SUT struct
su_rcot_data.V = @views [su_rcot_data.V; V3_alt1]
su_rcot_data.U = @views [su_rcot_data.U U3_alt1]
su_rcot_data.F = @views [su_rcot_data.F F3_alt1];

In [22]:
# Run the primal SU-RCOT in its absolute form
su_rcot_model = su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 123.16730999233869
value.(var_con) = [0.0, 0.9886952858964855, 0.9551106838981691, 1.1370613780710617, 1.0200016155581022, 1.0675512918734902]
value.(demand_con) = [51.0, 42.0, 31.0]
value.(factor_con) = [27.499999999999996, 6.666666666666666, 8.010912180897126, 21.208498620912597]


We can see that technology substitution occurs.

Once we change the availability of factors or the final demand for commodities, yet again other technologies are activated. For example, let us first increase final demand for commodity #c.3.

In [34]:
su_rcot_data.e[3] = 35

35

In [35]:
# Run the primal SU-RCOT in its absolute form
su_rcot_model = su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 128.0
value.(var_con) = [0.6454643903939872, 1.1313268403964185, 1.1667499453467858, 0.4494151518954526, 0.9206557232037282]
value.(demand_con) = [51.0, 42.0, 35.0]
value.(factor_con) = [26.608650920805463, 7.069999999999999, 8.479673664827201, 22.725]


Let us now increase in addition the availability of factors #f.2 and #f.4.

In [ ]:
su_rcot_data.e[3] = 35

35

In [ ]:
# Run the primal SU-RCOT in its absolute form
su_rcot_model = su_rcot("primal", "abs", su_rcot_data);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 128.0
value.(var_con) = [0.6454643903939872, 1.1313268403964185, 1.1667499453467858, 0.4494151518954526, 0.9206557232037282]
value.(demand_con) = [51.0, 42.0, 35.0]
value.(factor_con) = [26.608650920805463, 7.069999999999999, 8.479673664827201, 22.725]


Back to [table of contents](#toc)
<a id='add_jp_3mp_jp'></a>